In [ ]:
# ! pip install git+https://github.com/catalyst-team/catalyst@kittylyst scikit-learn>=0.20 optuna --upgrade

In [ ]:
# ! pip install catalyst==21.02rc1 scikit-learn>=0.20 optuna --upgrade

In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#  Catalyst 21.xx demo
## Stage 1: Customization is all u need
- 10 minimal examples with different Catalyst customization usecases

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_blobs
%matplotlib inline

In [ ]:
from typing import *

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset

from catalyst import dl, metrics, utils

In [ ]:
# make up a dataset
def make_dataset(seed=42, n_samples=int(1e3)):
    np.random.seed(seed)
    random.seed(seed)
    X, y = make_moons(n_samples=n_samples, noise=0.1)

    y = y*2 - 1 # make y be -1 or 1
    return X, y

def visualize_dataset(X, y):
    plt.figure(figsize=(5,5))
    plt.scatter(X[:,0], X[:,1], c=y, s=20, cmap='jet')

# let's create train data
X_train, y_train = make_dataset()
visualize_dataset(X_train, y_train)

In [ ]:
# valid data
X_valid, y_valid = make_dataset(seed=137)
visualize_dataset(X_valid, y_valid)

In [ ]:
# and another train one (why not?)
X_train2, y_train2 = make_dataset(seed=1337)
visualize_dataset(X_train2, y_train2)

In [ ]:
# initialize a model 
# 2-layer neural network
model = nn.Sequential(
    nn.Linear(2, 16), nn.ReLU(), 
    nn.Linear(16, 16), nn.ReLU(), 
    nn.Linear(16, 1)
)
print(model)
# print("number of parameters", len(model.parameters()))

In [ ]:
def visualize_decision_boundary(X, y, model):
    h = 0.25
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Xmesh = np.c_[xx.ravel(), yy.ravel()]
    
    inputs = torch.tensor([list(xrow) for xrow in Xmesh]).float()
    scores = model(inputs)
    
    Z = np.array([s.data > 0 for s in scores])
    Z = Z.reshape(xx.shape)

    fig = plt.figure()
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.show()
    return fig

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

In [ ]:
t1 = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train > 0).float())
t2 = TensorDataset(torch.tensor(X_train2).float(), torch.tensor(y_train2 > 0).float())
v1 = TensorDataset(torch.tensor(X_valid).float(), torch.tensor(y_valid > 0).float())

loaders = {
    "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
    "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
    "valid": DataLoader(v1, batch_size=32, num_workers=1), 
}

---

### Act 1 - ``CustomRunner – batch handling by you own``

In [ ]:
class CustomRunner(dl.IStageBasedRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str, epoch: int = None) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str, epoch: int = None):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str, epoch: int = None):
        return None

    def get_optimizer(self, model, stage: str, epoch: int = None):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, optimizer, stage: str, epoch: int = None):
        return None
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)

        loss = F.binary_cross_entropy_with_logits(y_hat.view(-1), y)
        self.batch_metrics = {"loss": loss}
        if self.loader_batch_step % 10 == 0:
            print(
                f"{self.loader_key} ({self.loader_batch_step}/{self.loader_batch_len}:" 
                f"loss {loss.item()}"
            )

        if self.is_train_loader:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

runner = CustomRunner().run()
model = runner.model

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 2 - ``SupervisedRunner – Runner with Callbacks``

In [ ]:
class CustomSupervisedRunner(dl.IStageBasedRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
    #         "csv": dl.LogdirLogger(logdir="./logdir02"),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir02/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str, epoch: int = None) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str, epoch: int = None):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str, epoch: int = None):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, model, stage: str, epoch: int = None):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, optimizer, stage: str, epoch: int = None):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str, epoch: int = None):
        return {
            # Let's use AUC metric as an example – it's loader-based, so we shouldn't compute it on each batch
            "auc": dl.LoaderMetricCallback(
                metric=metrics.AUCMetric(),
                input_key="scores", target_key="targets", 
            ), 
            # To wrap the criterion step logic, you could use CriterionCallback:
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            # To wrap the optimizer step logic, you could use OptimizerCallback:
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            # The same case with the scheduler:
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            # We could also use lrfinder for lr scheduling:
#             "lr-finder": dl.LRFinder(
#                 final_lr=1.0,
#                 scale="log",
#                 num_steps=None,
#                 optimizer_key=None,
#             ),
            # You can select any number of metrics to checkpoint on:
            "checkpoint1": dl.CheckpointCallback(
                logdir="./logdir02/auc",
                loader_key="valid", metric_key="auc", 
                minimize=False, save_n_best=3
            ),
            "checkpoint2": dl.CheckpointCallback(
                logdir="./logdir02/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
            # Or turn on/off tqdm verbose during loader run:
            "verbose": dl.TqdmCallback(),
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 3 - ``CustomMetric``

In [ ]:
class CustomAccuracyMetric(metrics.ICallbackBatchMetric, metrics.AdditiveValueMetric):
    def update(self, scores: torch.Tensor, targets: torch.Tensor) -> float:
        value = ((scores > 0.5) == targets).float().mean().item()
        value = super().update(value, len(targets))
        return value
    
    def update_key_value(self, scores: torch.Tensor, targets: torch.Tensor) -> Dict[str, float]:
        value = self.update(scores, targets)
        return {"accuracy": value}

    def compute_key_value(self) -> Dict[str, float]:
        mean, std = super().compute()
        return {"accuracy": mean, "accuracy/std": std}

    
class CustomSupervisedRunner(dl.IStageBasedRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir03/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str, epoch: int = None) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str, epoch: int = None):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str, epoch: int = None):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, model, stage: str, epoch: int = None):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, optimizer, stage: str, epoch: int = None):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str, epoch: int = None):
        return {
            "accuracy": dl.BatchMetricCallback(
                metric=CustomAccuracyMetric(), log_on_batch=True,
                input_key="scores", target_key="targets", 
            ),
            "auc": dl.LoaderMetricCallback(
                metric=metrics.AUCMetric(),
                input_key="scores", target_key="targets", 
            ), 
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint1": dl.CheckpointCallback(
                logdir="./logdir03/accuracy",
                loader_key="valid", metric_key="accuracy", 
                minimize=False, save_n_best=3
            ),
            "checkpoint2": dl.CheckpointCallback(
                logdir="./logdir03/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
    #         "verbose": dl.TqdmCallback(),
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 4 - ``CustomCallback``

In [ ]:
# Let's plot the decision doundary after each epoch:
class VisualizationCallback(dl.Callback):
    def __init__(self):
        super().__init__(order=dl.CallbackOrder.External)

    def on_epoch_end(self, runner):
        img = visualize_decision_boundary(X_valid, y_valid, runner.model)


class CustomSupervisedRunner(dl.IStageBasedRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir04/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str, epoch: int = None) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str, epoch: int = None):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str, epoch: int = None):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, model, stage: str, epoch: int = None):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, optimizer, stage: str, epoch: int = None):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str, epoch: int = None):
        return {
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint": dl.CheckpointCallback(
                logdir="./logdir04/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
            # And include it into callbacks:        
            "visualization": VisualizationCallback()
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

---

### Act 5 - ``CustomLogger``

In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
# We need to add only a few lines to log the image to all runner's loggers
class VisualizationCallback(dl.Callback):
    def __init__(self):
        super().__init__(order=dl.CallbackOrder.External)

    def on_epoch_end(self, runner):
        image = visualize_decision_boundary(X_valid, y_valid, runner.model)
        image = get_img_from_fig(image)
        # runner will propagate it to all loggers
        runner.log_image(tag="decision_boundary", image=image, scope="epoch")


# Let's also add our own Logger to store image on the disk
class VisualizationLogger(dl.ILogger):
    def __init__(self, logdir: str):
        self.logdir = logdir
        os.makedirs(self.logdir, exist_ok=True)
        
    def log_image(
        self,
        tag: str,
        image: np.ndarray,
        scope: str = None,
        # experiment info
        experiment_key: str = None,
        global_epoch_step: int = 0,
        global_batch_step: int = 0,
        global_sample_step: int = 0,
        # stage info
        stage_key: str = None,
        stage_epoch_len: int = 0,
        stage_epoch_step: int = 0,
        stage_batch_step: int = 0,
        stage_sample_step: int = 0,
        # loader info
        loader_key: str = None,
        loader_batch_len: int = 0,
        loader_sample_len: int = 0,
        loader_batch_step: int = 0,
        loader_sample_step: int = 0,
    ) -> None:
        if scope == "epoch":
            plt.imsave(
                os.path.join(self.logdir, f"{tag}_{stage_key}_{stage_epoch_step}.png"),
                image,
            )


class CustomSupervisedRunner(dl.IStageBasedRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "visualization": VisualizationLogger(logdir="./logdir05/visualization"),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir05/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str, epoch: int = None) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str, epoch: int = None):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str, epoch: int = None):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, model, stage: str, epoch: int = None):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, optimizer, stage: str, epoch: int = None):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str, epoch: int = None):
        return {
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint": dl.CheckpointCallback(
                logdir="./logdir05/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
            "visualization": VisualizationCallback()
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

In [ ]:
! ls ./logdir05
! ls ./logdir05/loss
! ls ./logdir05/tb
! ls ./logdir05/visualization

### Act 6 - ``Multistage Run``

In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
loaders = {
    "stage_1": {
        "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
        "valid": DataLoader(v1, batch_size=32, num_workers=1), 
    },
    "stage_2": {
        "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
        "valid": DataLoader(v1, batch_size=32, num_workers=1), 
    },
}

    
class VisualizationCallback(dl.Callback):
    def __init__(self):
        super().__init__(order=dl.CallbackOrder.External)

    def on_epoch_end(self, runner):
        image = visualize_decision_boundary(X_valid, y_valid, runner.model)
        image = get_img_from_fig(image)
        # runner will propagate it to all loggers
        runner.log_image(tag="decision_boundary", image=image, scope="epoch")


class VisualizationLogger(dl.ILogger):
    def __init__(self, logdir: str):
        self.logdir = logdir
        os.makedirs(self.logdir, exist_ok=True)
        
    def log_image(
        self,
        tag: str,
        image: np.ndarray,
        scope: str = None,
        # experiment info
        experiment_key: str = None,
        global_epoch_step: int = 0,
        global_batch_step: int = 0,
        global_sample_step: int = 0,
        # stage info
        stage_key: str = None,
        stage_epoch_len: int = 0,
        stage_epoch_step: int = 0,
        stage_batch_step: int = 0,
        stage_sample_step: int = 0,
        # loader info
        loader_key: str = None,
        loader_batch_len: int = 0,
        loader_sample_len: int = 0,
        loader_batch_step: int = 0,
        loader_sample_step: int = 0,
    ) -> None:
        if scope == "epoch":
            plt.imsave(
                os.path.join(self.logdir, f"{tag}_{stage_key}_{stage_epoch_step}.png"),
                image,
            )

class CustomSupervisedRunner(dl.IStageBasedRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "visualization": VisualizationLogger(logdir="./logdir06/visualization"),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir06/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return loaders.keys()
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str, epoch: int = None) -> "OrderedDict[str, DataLoader]":
        return loaders[stage]
    
    def get_model(self, stage: str, epoch: int = None):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str, epoch: int = None):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, model, stage: str, epoch: int = None):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, optimizer, stage: str, epoch: int = None):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str, epoch: int = None):
        return {
            "auc": dl.LoaderMetricCallback(
                metric=metrics.AUCMetric(),
                input_key="scores", target_key="targets", 
            ), 
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint1": dl.CheckpointCallback(
                logdir="./logdir06/auc",
                loader_key="valid", metric_key="auc", 
                minimize=False, save_n_best=3
            ),
            "checkpoint2": dl.CheckpointCallback(
                logdir="./logdir06/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
            "visualization": VisualizationCallback(),
    #         "verbose": TqdmCallback(),

        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

In [ ]:
! ls ./logdir06

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 7 - ``CustomRunner``

In [ ]:
class CustomSupervisedRunner(dl.IStageBasedRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir07/tb"),
        }
    
    @property
    def seed(self) -> int:
        return 73

    @property
    def name(self) -> str:
        return "experiment73"
    
    @property
    def stages(self) -> Iterable[str]:
        return ["stage_1", "stage_2"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str, epoch: int = None) -> "OrderedDict[str, DataLoader]":
        if stage == "stage_1":
            return {
                "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
                "valid": DataLoader(v1, batch_size=32, num_workers=1), 
            }
        elif stage == "stage_2":
            return {
                "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
                "valid": DataLoader(v1, batch_size=32, num_workers=1), 
            }
        else:
            raise NotImplemented()
    
    def get_model(self, stage: str, epoch: int = None):
        if self.model is not None:
            return self.model
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str, epoch: int = None):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, model, stage: str, epoch: int = None):
        if stage == "stage_1":
            return torch.optim.Adam(model.parameters(), lr=0.02)
        elif stage == "stage_2":
            return torch.optim.SGD(model.parameters(), lr=0.01)
        else:
            raise NotImplemented()

    def get_scheduler(self, optimizer, stage: str, epoch: int = None):
        if stage == "stage_1":
            return torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 8])
        elif stage == "stage_2":
            return torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 6])
        else:
            raise NotImplemented()
    
    def get_callbacks(self, stage: str, epoch: int = None):
        if stage == "stage_1":
            return {
                "criterion": dl.CriterionCallback(
                    metric_key="loss", 
                    input_key="logits", 
                    target_key="targets"
                ), 
                "optimizer": dl.OptimizerCallback(metric_key="loss"), 
                "scheduler": dl.SchedulerCallback(
                    loader_key="valid", metric_key="loss"
                ),
                "checkpoint": dl.CheckpointCallback(
                    logdir="./logdir07/loss",
                    loader_key="valid", metric_key="loss", 
                    minimize=True, save_n_best=3
                ),
            }
        elif stage == "stage_2":
            return {
                "auc": dl.LoaderMetricCallback(
                    metric=metrics.AUCMetric(),
                    input_key="scores", target_key="targets", 
                ), 
                "criterion": dl.CriterionCallback(
                    metric_key="loss", 
                    input_key="logits", 
                    target_key="targets"
                ), 
                "optimizer": dl.OptimizerCallback(metric_key="loss"), 
                "scheduler": dl.SchedulerCallback(
                    loader_key="valid", metric_key="loss"
                ),
                "checkpoint_auc": dl.CheckpointCallback(
                    logdir="./logdir07/auc",
                    loader_key="valid", metric_key="auc", 
                    minimize=False, save_n_best=3
                ),
            }
        else:
            raise NotImplemented()
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

---

### Act 8 - integration with hyperparameter search

In [ ]:
from datetime import datetime
import optuna    

def objective(trial):
    num_epochs = 6
    num_hidden1 = int(trial.suggest_loguniform("num_hidden1", 2, 16))
    num_hidden2 = int(trial.suggest_loguniform("num_hidden2", 2, 16))
    logdir = f"./logdir08/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    
    loaders = {
        "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
        "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
        "valid": DataLoader(v1, batch_size=32, num_workers=1), 
    }

    class CustomRunner(dl.IStageBasedRunner):
        def get_trial(self):
            return trial

        def get_engine(self) -> dl.IEngine:
            return dl.DeviceEngine("cpu")

        def get_loggers(self):
            return {
                "console": dl.ConsoleLogger(),
                "tensorboard": dl.TensorboardLogger(logdir=f"{logdir}/tb"),
            }

        @property
        def seed(self) -> int:
            return 73

        @property
        def name(self) -> str:
            return "experiment73"

        @property
        def stages(self) -> Iterable[str]:
            return ["stage_1", "stage_2"]

        def get_stage_len(self, stage: str) -> int:
            return num_epochs

        def get_loaders(self, stage: str, epoch: int = None) -> "OrderedDict[str, DataLoader]":
            return loaders

        def get_model(self, stage: str, epoch: int = None):
            return nn.Sequential(
                nn.Linear(2, num_hidden1), nn.ReLU(), 
                nn.Linear(num_hidden1, num_hidden2), nn.ReLU(), 
                nn.Linear(num_hidden2, 1)
            )

        def get_criterion(self, stage: str, epoch: int = None):
            return nn.BCEWithLogitsLoss()

        def get_optimizer(self, model, stage: str, epoch: int = None):
            return torch.optim.Adam(model.parameters(), lr=0.02)

        def get_scheduler(self, optimizer, stage: str, epoch: int = None):
            return torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 6])

        def get_callbacks(self, stage: str, epoch: int = None):
            return {
                "auc": dl.LoaderMetricCallback(
                    metric=metrics.AUCMetric(),
                    input_key="scores", target_key="targets", 
                ), 
                "criterion": dl.CriterionCallback(
                    metric_key="loss", 
                    input_key="logits", 
                    target_key="targets"
                ), 
                "optimizer": dl.OptimizerCallback(metric_key="loss"), 
                "scheduler": dl.SchedulerCallback(
                    loader_key="valid", metric_key="loss"
                ),
                "checkpoint": dl.CheckpointCallback(
                    logdir=f"{logdir}/auc",
                    loader_key="valid", metric_key="auc", 
                    minimize=False, save_n_best=3
                ),
                "optuna": dl.OptunaPruningCallback(loader_key="valid", metric_key="auc", minimize=False)
            }

        def handle_batch(self, batch):
            x, y = batch
            y_hat = self.model(x)

            self.batch = {
                "features": x,
                "targets": y,
                "logits": y_hat.view(-1),
                "scores": torch.sigmoid(y_hat.view(-1)),
            }

    runner = CustomRunner()
    runner.run()
    score = runner.callbacks["optuna"].best_score
    
    return score

study = optuna.create_study(
    direction="maximize",
#     direction="minimize",
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=0, n_warmup_steps=0, interval_steps=1
    ),
)
study.optimize(objective, n_trials=5, timeout=300)
print(study.best_value, study.best_params)

---

### Act 9 - Confusion Matrix logging - IMetric+ICallback+ILogger

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst import dl, metrics, utils

# sample data
num_samples, num_features, num_classes = int(1e4), int(1e1), 6
num_epochs = 6

class CustomSupervisedRunner(dl.IStageBasedRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "csv": dl.CSVLogger(logdir="./logdir09"),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir09/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return num_epochs
    
    def get_loaders(self, stage: str, epoch: int = None) -> "OrderedDict[str, DataLoader]":
        # sample data
        num_samples, num_features, num_classes = int(1e4), int(1e1), 6
        X = torch.rand(num_samples, num_features)
        y = (torch.rand(num_samples, ) * num_classes).to(torch.int64)

        # pytorch loaders
        dataset = TensorDataset(X, y)
        loader = DataLoader(dataset, batch_size=32, num_workers=1)
        loaders = {"train": loader, "valid": loader}
        return loaders
    
    def get_model(self, stage: str, epoch: int = None):
        return torch.nn.Linear(num_features, num_classes)

    def get_criterion(self, stage: str, epoch: int = None):
        return torch.nn.CrossEntropyLoss()

    def get_optimizer(self, model, stage: str, epoch: int = None):
        return torch.optim.Adam(model.parameters())

    def get_scheduler(self, optimizer, stage: str, epoch: int = None):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2])
    
    def get_callbacks(self, stage: str, epoch: int = None):
        return {
            "accuracy": dl.BatchMetricCallback(
                metric=metrics.AccuracyMetric(num_classes=num_classes),
                input_key="probs", target_key="targets", 
            ),
            "auc": dl.LoaderMetricCallback(
                metric=metrics.AUCMetric(),
                input_key="scores", target_key="targets", 
            ), 
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets",
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint1": dl.CheckpointCallback(
                logdir="./logdir09/loss",
                loader_key="valid", metric_key="loss", 
                minimize=False, save_n_best=3
            ),
            "checkpoint2": dl.CheckpointCallback(
                logdir="./logdir09/auc",
                loader_key="valid", metric_key="auc", 
                minimize=True, save_n_best=1
            ),
            "checkpoint3": dl.CheckpointCallback(
                logdir="./logdir9/accuracy",
                loader_key="valid", metric_key="accuracy", 
                minimize=True, save_n_best=1
            ),
            "verbose": dl.TqdmCallback(),
            "confusion_matrix": dl.ConfusionMatrixCallback(
                input_key="probs", 
                target_key="targets",
                prefix="confusion_matrix",
                num_classes=num_classes,
            )
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat,
            "scores": torch.sigmoid(y_hat),
            "probs": torch.softmax(y_hat, dim=1),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

---

In [ ]:
### Act 10 - @TODO

---

## Stage 2: PythonAPI is all u need
- 10 minimal examples with different Catalyst user-friendly PythonAPI usecases

In [ ]:
# let's start minimal examples section
from catalyst import dl, metrics, utils

### Act 11 - ML - linear regression

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst import dl


# data
num_samples, num_features = int(1e4), int(1e1)
X, y = torch.rand(num_samples, num_features), torch.rand(num_samples)
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, num_workers=1)
loaders = {"train": loader, "valid": loader}

# model, criterion, optimizer, scheduler
model = torch.nn.Linear(num_features, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 6])

# model training
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets"
)
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir="./logdir11",
    num_epochs=8,
    verbose=True,
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
)

---

### Act 12 - ML - multiclass classification

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst import dl, metrics, utils

# sample data
num_samples, num_features, num_classes = int(1e4), int(1e1), 4
X = torch.rand(num_samples, num_features)
y = (torch.rand(num_samples, ) * num_classes).to(torch.int64)

# pytorch loaders
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, num_workers=1)
loaders = {"train": loader, "valid": loader}

# model, criterion, optimizer, scheduler
model = torch.nn.Linear(num_features, num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2])

# model training
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets"
)
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir="./logdir12",
    num_epochs=6,
    verbose=True,
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    callbacks=[dl.AccuracyCallback(input_key="logits", target_key="targets", num_classes=num_classes)]
#     callbacks={
#         "classification": dl.BatchMetricCallback(
#             metric=metrics.MulticlassPrecisionRecallF1SupportMetric(num_classes=num_classes),
#             input_key="logits", target_key="targets", 
#         ),
#     },
)

----

### Act 13 - ML - multilabel classification

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst import dl

# sample data
num_samples, num_features, num_classes = int(1e4), int(1e1), 4
X = torch.rand(num_samples, num_features)
y = (torch.rand(num_samples, num_classes) > 0.5).to(torch.float32)

# pytorch loaders
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, num_workers=1)
loaders = {"train": loader, "valid": loader}

# model, criterion, optimizer, scheduler
model = torch.nn.Linear(num_features, num_classes)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2])

# model training
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets"
)
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir="./logdir",
    num_epochs=3,
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    callbacks={
        "classification": dl.BatchMetricCallback(
            metric=metrics.MultilabelPrecisionRecallF1SupportMetric(num_classes=num_classes),
            input_key="logits", target_key="targets", 
        ),
    },
)

---

In [ ]:
### Act 14 - CV - MNIST classification

In [ ]:
### Act 15 - CV - classification with AutoEncoder

In [ ]:
### Act 16 - CV - classification with Variational AutoEncoder

### Act 17 - CV - segmentation with classification auxiliary task

In [ ]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from catalyst import dl, metrics
from catalyst.contrib.data.cv import ToTensor
from catalyst.contrib.datasets import MNIST

class ClassifyUnet(nn.Module):

    def __init__(self, in_channels, in_hw, out_features):
        super().__init__()
        self.encoder = nn.Sequential(nn.Conv2d(in_channels, in_channels, 3, 1, 1), nn.Tanh())
        self.decoder = nn.Conv2d(in_channels, in_channels, 3, 1, 1)
        self.clf = nn.Linear(in_channels * in_hw * in_hw, out_features)

    def forward(self, x):
        z = self.encoder(x)
        z_ = z.view(z.size(0), -1)
        y_hat = self.clf(z_)
        x_ = self.decoder(z)
        return y_hat, x_

model = ClassifyUnet(1, 28, 10)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)

loaders = {
    "train": DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()), batch_size=32),
    "valid": DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()), batch_size=32),
}

class CustomRunner(dl.Runner):

    def handle_batch(self, batch):
        x, y = batch
        x_noise = (x + torch.rand_like(x)).clamp_(0, 1)
        y_hat, x_ = self.model(x_noise)
        
        self.batch = {
#             "image": x,
            "clf_targets": y,
            "seg_targets": x,
            "clf_logits": y_hat,
            "seg_logits": x_,
        }


runner = CustomRunner()
runner.train(
    loaders=loaders, 
    model=model, 
    criterion=criterion,
    optimizer=optimizer, 
    logdir="./logdir14",
    num_epochs=6,
    verbose=True,
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    callbacks={
        "classification": dl.BatchMetricCallback(
            metric=metrics.MulticlassPrecisionRecallF1SupportMetric(num_classes=10),
            input_key="clf_logits", target_key="clf_targets", 
        ),
        "segmentation": dl.BatchMetricCallback(
            metric=metrics.IOUMetric(),
            input_key="seg_logits", target_key="seg_targets", 
        ),
        "criterion": dl.CriterionCallback(
            metric_key="loss", 
            input_key="clf_logits", 
            target_key="clf_targets",
        ), 
        "optimizer": dl.OptimizerCallback(metric_key="loss"), 
    },
)

---

### Act 18 - CV - MNIST with Metric Learning

In [ ]:
# from torch.optim import Adam
# from torch.utils.data import DataLoader

# from catalyst import data, dl, utils
# from catalyst.contrib import datasets, models, nn
# import catalyst.contrib.data.cv.transforms.torch as t


# # 1. train and valid datasets
# dataset_root = "."
# transforms = t.Compose([t.ToTensor(), t.Normalize((0.1307,), (0.3081,))])

# dataset_train = datasets.MnistMLDataset(root=dataset_root, download=True, transform=transforms)
# sampler = data.BalanceBatchSampler(labels=dataset_train.get_labels(), p=5, k=10)
# train_loader = DataLoader(dataset=dataset_train, sampler=sampler, batch_size=sampler.batch_size)

# dataset_val = datasets.MnistQGDataset(root=dataset_root, transform=transforms, gallery_fraq=0.2)
# val_loader = DataLoader(dataset=dataset_val, batch_size=1024)

# # 2. model and optimizer
# model = models.SimpleConv(features_dim=16)
# optimizer = Adam(model.parameters(), lr=0.001)

# # 3. criterion with triplets sampling
# sampler_inbatch = data.HardTripletsSampler(norm_required=False)
# criterion = nn.TripletMarginLossWithSampler(margin=0.5, sampler_inbatch=sampler_inbatch)

# # 4. training with catalyst Runner
# callbacks = [
#     dl.ControlFlowCallback(
#         dl.CriterionCallback(input_key="logits", target_key="targets", metric_key="loss"), 
#         loaders="train"
#     ),
#     dl.ControlFlowCallback(dl.CMCScoreCallback(topk_args=[1]), loaders="valid"),
#     dl.PeriodicLoaderCallback(valid=100),
# ]

# runner = dl.SupervisedRunner(
#     input_key="features", output_key="logits", target_key="targets"
# )
# runner.train(
#     model=model,
#     criterion=criterion,
#     optimizer=optimizer,
#     callbacks=callbacks,
#     loaders={"train": train_loader, "valid": val_loader},
#     minimize_metric=False,
#     verbose=True,
#     valid_loader="valid",
#     num_epochs=200,
#     main_metric="cmc01",
# )   

---

### Act 19 - GAN - MNIST, flatten version

In [ ]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from catalyst import dl
from catalyst.contrib.data.cv import ToTensor
from catalyst.contrib.datasets import MNIST
from catalyst.contrib.nn.modules import Flatten, GlobalMaxPool2d, Lambda

latent_dim = 128
generator = nn.Sequential(
    # We want to generate 128 coefficients to reshape into a 7x7x128 map
    nn.Linear(128, 128 * 7 * 7),
    nn.LeakyReLU(0.2, inplace=True),
    Lambda(lambda x: x.view(x.size(0), 128, 7, 7)),
    nn.ConvTranspose2d(128, 128, (4, 4), stride=(2, 2), padding=1),
    nn.LeakyReLU(0.2, inplace=True),
    nn.ConvTranspose2d(128, 128, (4, 4), stride=(2, 2), padding=1),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(128, 1, (7, 7), padding=3),
    nn.Sigmoid(),
)
discriminator = nn.Sequential(
    nn.Conv2d(1, 64, (3, 3), stride=(2, 2), padding=1),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(64, 128, (3, 3), stride=(2, 2), padding=1),
    nn.LeakyReLU(0.2, inplace=True),
    GlobalMaxPool2d(),
    Flatten(),
    nn.Linear(128, 1)
)

model = {"generator": generator, "discriminator": discriminator}
optimizer = {
    "generator": torch.optim.Adam(generator.parameters(), lr=0.0003, betas=(0.5, 0.999)),
    "discriminator": torch.optim.Adam(discriminator.parameters(), lr=0.0003, betas=(0.5, 0.999)),
}
loaders = {
    "train": DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()), batch_size=32),
}

class CustomRunner(dl.Runner):

    def handle_batch(self, batch):
        real_images, _ = batch
        batch_metrics = {}
        
        # Sample random points in the latent space
        batch_size = real_images.shape[0]
        random_latent_vectors = torch.randn(batch_size, latent_dim).to(self.device)
        
        # Decode them to fake images
        generated_images = self.model["generator"](random_latent_vectors).detach()
        # Combine them with real images
        combined_images = torch.cat([generated_images, real_images])
        
        # Assemble labels discriminating real from fake images
        labels = torch.cat([
            torch.ones((batch_size, 1)), torch.zeros((batch_size, 1))
        ]).to(self.device)
        # Add random noise to the labels - important trick!
        labels += 0.05 * torch.rand(labels.shape).to(self.device)
        
        # Train the discriminator
        predictions = self.model["discriminator"](combined_images)
        batch_metrics["loss_discriminator"] = \
          F.binary_cross_entropy_with_logits(predictions, labels)
        
        # Sample random points in the latent space
        random_latent_vectors = torch.randn(batch_size, latent_dim).to(self.device)
        # Assemble labels that say "all real images"
        misleading_labels = torch.zeros((batch_size, 1)).to(self.device)
        
        # Train the generator
        generated_images = self.model["generator"](random_latent_vectors)
        predictions = self.model["discriminator"](generated_images)
        batch_metrics["loss_generator"] = \
          F.binary_cross_entropy_with_logits(predictions, misleading_labels)
        
        self.batch_metrics.update(**batch_metrics)

runner = CustomRunner()
runner.train(
    model=model, 
    optimizer=optimizer,
    loaders=loaders,
    callbacks=[
        dl.OptimizerCallback(
            model_key="generator",
            optimizer_key="generator", 
            metric_key="loss_generator"
        ),
        dl.OptimizerCallback(
            model_key="discriminator", 
            optimizer_key="discriminator", 
            metric_key="loss_discriminator"
        ),
    ],
#     valid_loader="train",
#     valid_metric="loss_generator",
#     minimize_valid_metric=True,
    num_epochs=1,
    verbose=True,
#     logdir="./logdir19",
)

---

### Act 20 - AutoML - hyperparameters optimization with Optuna

In [ ]:
import os
import optuna
import torch
from torch import nn
from torch.utils.data import DataLoader
from catalyst import dl
from catalyst.contrib.data.cv import ToTensor
from catalyst.contrib.datasets import MNIST
from catalyst.contrib.nn import Flatten
    

def objective(trial):
    lr = trial.suggest_loguniform("lr", 1e-3, 1e-1)
    num_hidden = int(trial.suggest_loguniform("num_hidden", 32, 128))

    loaders = {
        "train": DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()), batch_size=32),
        "valid": DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()), batch_size=32),
    }
    model = nn.Sequential(
        Flatten(), nn.Linear(784, num_hidden), nn.ReLU(), nn.Linear(num_hidden, 10)
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    runner = dl.SupervisedRunner(
        input_key="features", output_key="logits", target_key="targets"
    )
    runner.train(
        model=model,
        loaders=loaders,
        criterion=criterion,
        optimizer=optimizer,
        callbacks={
            "optuna": dl.OptunaPruningCallback(loader_key="valid", metric_key="accuracy01", minimize=False, trial=trial),
            "accuracy": dl.AccuracyCallback(input_key="logits", target_key="targets", num_classes=10),
        },
        num_epochs=10,
#         valid_loader="valid",
#         valid_metric="accuracy01",
#         minimize_valid_metric=False,
    )
    score = runner.callbacks["optuna"].best_score
    return score

study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=1, n_warmup_steps=0, interval_steps=1
    ),
)
study.optimize(objective, n_trials=10, timeout=300)
print(study.best_value, study.best_params)

----

🎉 You have passed ``Kittylyst`` tutorial! This is just a minimal educational demo, but I hope you found it interesting for your deep learning research code organisation.

For more advanced and production-ready solution please follow our [Catalyst](https://github.com/catalyst-team/catalyst) repository.

PS. If you are interested in deep learning you could also try out our [dl-course](https://github.com/catalyst-team/dl-course).